# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [166]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf

In [167]:
previsao_renda_df = pd.read_csv('previsao_de_renda.csv')
previsao_renda_df.sample(5)


,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
14400,14400,2016-03-01,12136,F,True,False,1,Assalariado,Secundário,União,Casa,33,5.780822,3.0,7168.75
13961,13961,2016-02-01,8917,F,False,False,1,Assalariado,Superior completo,União,Casa,32,9.049315,3.0,3814.21
11776,11776,2015-12-01,2612,F,True,True,0,Assalariado,Secundário,Casado,Casa,51,1.306849,2.0,1754.46
8538,8538,2015-09-01,202,F,False,False,0,Servidor público,Superior completo,Casado,Casa,40,15.282192,2.0,2402.25
14333,14333,2016-03-01,6399,F,True,False,0,Assalariado,Secundário,Casado,Casa,38,20.175342,2.0,1636.88


In [168]:
previsao_renda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [169]:
previsao_renda_df.dropna(inplace=True)
previsao_renda_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

In [170]:
previsao_renda_df.drop(columns=['Unnamed: 0'], inplace=True)
previsao_renda_df.drop(columns=['data_ref'], inplace=True)

In [171]:
previsao_renda_df.columns

Index(['id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda'],
      dtype='object')

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [172]:
previsao_renda_df["posse_de_veiculo"] = previsao_renda_df["posse_de_veiculo"].astype(int)
previsao_renda_df["posse_de_imovel"] = previsao_renda_df["posse_de_imovel"].astype(int)
previsao_renda_df = pd.get_dummies(
    previsao_renda_df,
    columns=['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'],
    drop_first=True)

X = previsao_renda_df.drop(columns=['renda']).copy()
y = previsao_renda_df[['renda']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, random_state=42)

regularization_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

In [173]:
for alpha in regularization_alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)

    y_pred_ridge = ridge.predict(X_test)

    r2_ridge = r2_score(y_test, y_pred_ridge)
    print(f"O R-quadrado para Ridge com o Alpha iguail a {alpha} é {r2_ridge}")

O R-quadrado para Ridge com o Alpha iguail a 0 é 0.2980913923090285
O R-quadrado para Ridge com o Alpha iguail a 0.001 é 0.2980914584843043
O R-quadrado para Ridge com o Alpha iguail a 0.005 é 0.2980917230455732
O R-quadrado para Ridge com o Alpha iguail a 0.01 é 0.2980920534330559
O R-quadrado para Ridge com o Alpha iguail a 0.05 é 0.29809468407037387
O R-quadrado para Ridge com o Alpha iguail a 0.1 é 0.2980979416948465


In [176]:
for alpha in regularization_alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    y_pred_lasso = lasso.predict(X_test)

    r2_lasso = r2_score(y_test, y_pred_lasso)
    print(f"O R-quadrado para Lasso com o Alpha iguail a {alpha} é {r2_lasso}")

/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently 

O R-quadrado para Lasso com o Alpha iguail a 0 é 0.2980917689607735
O R-quadrado para Lasso com o Alpha iguail a 0.001 é 0.29809224852379235


/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.950e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.588e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


O R-quadrado para Lasso com o Alpha iguail a 0.005 é 0.29809416413230605


/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.104e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


O R-quadrado para Lasso com o Alpha iguail a 0.01 é 0.29809655269500934


/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.970e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


O R-quadrado para Lasso com o Alpha iguail a 0.05 é 0.29811542327876495
O R-quadrado para Lasso com o Alpha iguail a 0.1 é 0.29813841671384045


/home/thiago/Desenvolvimento/estudos_data_science/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.764e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


In [175]:
# Regularization
modelo = """renda ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda +
            educacao + estado_civil + tipo_residencia + idade + tempo_emprego + 
            C(qt_pessoas_residencia)"""
md = smf.ols(modelo, data = X_test)

reg = md.fit_regularized(method = 'elastic_net' 
                        , refit = True
                        , L1_wt = 0  # com zero indica que é regularização do tipo Ridge
                        , alpha = 0.01)

reg.sumary()

# for alpha in regularization_alphas:
#     reg = md.fit_regularized(method = 'elastic_net' 
#                             , refit = True
#                             , L1_wt = 1
#                             , alpha = alpha)

#     print(f"O R-quadrado para Ridge com o Alpha iguail a {alpha} é {reg.rsquared}")
#     y_pred = reg.predict(X_test)

PatsyError: Error evaluating factor: NameError: name 'tipo_residencia' is not defined
    renda ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda +             educacao + estado_civil + tipo_residencia + idade + tempo_emprego +              C(qt_pessoas_residencia)
                                                                                                                           ^^^^^^^^^^^^^^^